In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [44.4 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-01 00:39:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-02-01 00:39:01 (11.5 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load Amazon Data (from Deliverable 1)
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

**Deliverable 2: Determine Bias of Vine Reviews**

In [8]:
# Step 1:
# Filter the data and create a new DataFrame or table to retrieve all the rows 
# where the total_votes count is equal to or greater than 20 to pick reviews 
# that are more likely to be helpful and to avoid having division by zero errors 
# later on.
total_votes_df = vine_df.filter("total_votes>=20")
total_votes_df.show()
print(total_votes_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [9]:
# Step 2:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame
# or table to retrieve all the rows where the number of helpful_votes divided by
# total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.filter("helpful_votes/total_votes>=.5")
helpful_votes_df.show()
print(helpful_votes_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [10]:
# Step 3:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or
# table that retrieves all the rows where a review was written as part of the 
# Vine program (paid), vine == 'Y'.
paid_vine_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
paid_vine_df.show()
print(paid_vine_df.count())

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+

0


In [11]:
# Step 4:
# Repeat Step 3, but this time retrieve all the rows where the review was not 
# part of the Vine program (unpaid), vine == 'N'.
unpaid_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
unpaid_review_df.show()
print(unpaid_review_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [17]:
# Step 5:
# Determine the total number of reviews, the number of 5-star reviews, and the 
# percentage of 5-star reviews for the two types of review (paid vs unpaid).

## Total Number of reviews for the two types of review (already done above):
print("Number of paid reviews (part of the Vine Program): %f" % paid_vine_df.count())
print("Number of unpaid reviews (reviews are not from the Vine Program): %f" % unpaid_review_df.count())


## Number of 5-star reviews for the two types of review:
paid_vine_5star_df = paid_vine_df.filter(paid_vine_df["star_rating"] == "5")
print("Number of 5-star of paid reviews: %f" % paid_vine_5star_df.count())

unpaid_reviews_5star_df = unpaid_review_df.filter(unpaid_review_df["star_rating"] == "5")
print("Number of 5-star of unpaid reviews: %f" % unpaid_reviews_5star_df.count())


## Percentage of 5-star reviews for the two types of review:
percent_paid = (paid_vine_5star_df.count()/helpful_votes_df.count())*100
print("Percentage of 5-star reviews from the paid vine program: %f" % percent_paid)

percent_unpaid = (unpaid_reviews_5star_df.count()/helpful_votes_df.count())*100
print("Percentage of 5-star reviews from the unpaid reviews: %f" % percent_unpaid)

Number of paid reviews (part of the Vine Program): 0.000000
Number of unpaid reviews (reviews are not from the Vine Program): 403807.000000
Number of 5-star of paid reviews: 0.000000
Number of 5-star of unpaid reviews: 242889.000000
Percentage of 5-star reviews from the paid vine program: 0.000000
Percentage of 5-star reviews from the unpaid reviews: 60.149775


### Deliverable 3: Additional Analysis

In [19]:
# Filter the data for unpaid reviews that are also verified purchases:
verified_purchase = unpaid_review_df.filter(unpaid_review_df["verified_purchase"] == "Y")
print("Number of reviews from verified purchases: %f" % verified_purchase.count())
print("Percentage of verified purchase reviews: %f" % (verified_purchase.count()/unpaid_review_df.count()))

unverified_purchase= unpaid_review_df.filter(unpaid_review_df["verified_purchase"] == "N")
print("Number of reviews from unverified purchases: %f" % unverified_purchase.count())
print("Percentage of unverified purchase reviews: %f" % (unverified_purchase.count()/unpaid_review_df.count()))

Number of reviews from verified purchases: 46200.000000
Percentage of verified purchase reviews: 0.114411
Number of reviews from unverified purchases: 357607.000000
Percentage of unverified purchase reviews: 0.885589


In [40]:
# Compare the ratings against verified and unverified purchases
## Create a verified and unverified purchase vs rating dataframes
verified_vs_rating = verified_purchase.groupby("star_rating").count()
unverified_vs_rating = unverified_purchase.groupby("star_rating").count()

## Change columns names
verified = verified_vs_rating.selectExpr('star_rating as Star_Rating', 'count as Verified_Purchase_Count')
unverified = unverified_vs_rating.selectExpr('star_rating as Star_Rating', 'count as Unverified_Purchase_Count')

## Join and order dataframe
df = verified.join(unverified, on="Star_Rating")
df.orderBy(df["Star_Rating"].desc()).show()

+-----------+-----------------------+-------------------------+
|Star_Rating|Verified_Purchase_Count|Unverified_Purchase_Count|
+-----------+-----------------------+-------------------------+
|          5|                  27495|                   215394|
|          4|                   7476|                    53422|
|          3|                   4431|                    26956|
|          2|                   3342|                    22744|
|          1|                   3456|                    39091|
+-----------+-----------------------+-------------------------+



+-----------+-----------------------+-------------------------+
|Star_Rating|Verified_Purchase_Count|Unverified_Purchase_Count|
+-----------+-----------------------+-------------------------+
|          1|                   3456|                    39091|
|          3|                   4431|                    26956|
|          5|                  27495|                   215394|
|          4|                   7476|                    53422|
|          2|                   3342|                    22744|
+-----------+-----------------------+-------------------------+

